In [2]:
#!/usr/bin/python
import cv2, os
import numpy as np
from PIL import Image
import csv
import pandas as pd
import pdb as pdb
from sklearn.cross_validation import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
import time

settings = {
    'minNeighbors': 2, 
    'minSize': (40,40)
}
image_list = pd.read_csv('driver_imgs_list.csv')
train_subject_subset = image_list.subject.value_counts()[-9:].index.values
test_subject_subset = image_list.subject.value_counts()[:-9].index.values
train = image_list[image_list.subject.isin(train_subject_subset)].reset_index(drop=True)
test = image_list[image_list.subject.isin(test_subject_subset)].reset_index(drop=True)

#take 25% of the sets 
mask = np.random.choice([False, True], len(train), p=[0.75, 0.25])
train = train[mask]
mask = np.random.choice([False, True], len(test), p=[0.75, 0.25])
test = test[mask]
print 'done getting train/test sets'
print 'getting images from paths'

done getting train/test sets
getting images from paths


/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
images = []
predict_images = []
labels = [] 
predict_labels = []

t0 = time.time()
scaled_size = (200,150)

for i, image in train.iterrows():
    image_path = './train/' + image.classname + '/' + image.img
    predict_image_pil = Image.open(image_path).convert('L') #greyscale
    predict_image_pil.thumbnail(scaled_size, Image.ANTIALIAS) #resize
    image = np.array(predict_image_pil, 'uint8') #to array
    images.append(image.flatten())
    labels.append(train['classname'][i])
    
for i, image in test.iterrows():
    image_path = './train/' + image.classname + '/' + image.img
    predict_image_pil = Image.open(image_path).convert('L') #greyscale
    predict_image_pil.thumbnail(scaled_size, Image.ANTIALIAS) #resize
    predict_image = np.array(predict_image_pil, 'uint8') #to array
    predict_images.append(predict_image.flatten())
    predict_labels.append(test['classname'][i])
print('done')



done


In [34]:
# Extract relevant data
training_data = images
training_labels = labels
prediction_data = predict_images
prediction_labels = predict_labels

training_labels = map(lambda each:int(each.strip("c")), training_labels)
prediction_labels = map(lambda each:int(each.strip("c")), prediction_labels)

training_data = map(lambda each:each.flatten(), training_data)
prediction_data = map(lambda each:each.flatten(), prediction_data)

print("done in {}s".format(time.time() - t0))
print ("training classifier")

done in 5045.3704741s
training classifier


In [35]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(training_data, training_labels)
pred = clf.predict(prediction_data)
correct = np.sum(pred == prediction_labels)
print 'accuracy using GradientBoostingClassifier: {} / {} %'.format(correct, len(prediction_labels))
print("done in {}s".format(time.time() - t0))


accuracy using GradientBoostingClassifier: 686 / 4085 %
done in 5633.85348105s


In [9]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(training_data, training_labels)
pred = clf.predict(prediction_data)
correct = np.sum(pred == prediction_labels)
print 'accuracy using RandomForestClassifier: {} / {} %'.format(correct, len(prediction_labels))
print("done in {}s".format(time.time() - t0))

accuracy using RandomForestClassifier: 919 / 4085 %
done in 1507.87428212s


In [21]:
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators=500, max_depth=None, 
                           min_samples_split=2, random_state=0,n_jobs=-1,warm_start=True)
clf = clf.fit(training_data, training_labels)
pred = clf.predict(prediction_data)
correct = np.sum(pred == prediction_labels)
print 'accuracy using ExtraTreesClassifier: {} / {} %'.format(correct, len(prediction_labels))

accuracy using ExtraTreesClassifier: 1173 / 4085 %


In [11]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2, random_state=0)
clf = clf.fit(training_data, training_labels)
pred = clf.predict(prediction_data)
correct = np.sum(pred == prediction_labels)
print 'accuracy using DecisionTreeClassifier: {} / {} %'.format(correct, len(prediction_labels))

accuracy using DecisionTreeClassifier: 622 / 4085 %


In [44]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=500,random_state=0,warm_start=True)#,learning_rate=)
clf = clf.fit(training_data, training_labels)
pred = clf.predict(prediction_data)
correct = np.sum(pred == prediction_labels)
print 'accuracy using AdaBoostClassifier: {} / {} %'.format(correct, len(prediction_labels))

TypeError: __init__() got an unexpected keyword argument 'warm_start'

In [41]:
from sklearn.semi_supervised import LabelPropagation

clf = LabelPropagation(kernel='rbf', gamma=20, n_neighbors=7, 
                       alpha=1, max_iter=30, tol=0.001, n_jobs=-1)#,learning_rate=)
clf = clf.fit(training_data, training_labels)
pred = clf.predict(prediction_data)
correct = np.sum(pred == prediction_labels)
print 'accuracy using LabelPropagation: {} / {} %'.format(correct, len(prediction_labels))

accuracy using LabelPropagation: 480 / 4085 %


/usr/local/lib/python2.7/site-packages/sklearn/semi_supervised/label_propagation.py:198: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer


In [45]:
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier(n_estimators=500, warm_start=True, n_jobs=-1)
clf = clf.fit(training_data, training_labels)
pred = clf.predict(prediction_data)
correct = np.sum(pred == prediction_labels)
print 'accuracy using BaggingClassifier: {} / {} %'.format(correct, len(prediction_labels))

accuracy using BaggingClassifier: 751 / 4085 %


In [6]:
# pdb.set_trace()
# t0 = time.time()
# correct = 0
# incorrect = 0
# for i, image in enumerate(prediction_data):
#     pred = OneVsOneClassifier(clf.predict(image))
#     if pred == prediction_labels[i]:
#         correct += 1
#     else:
#         incorrect += 1
# print 'accuracy using SVM one vs one: {}%'.format((100*correct)/(correct + incorrect))
# print("done in {}s".format(time.time() - t0))
# print ("making predictions")
    

In [7]:
# t0 = time.time()
# correct = 0
# incorrect = 0
# pred = OneVsOneClassifier(clf.predict(prediction_data))

# correct = np.sum(pred == prediction_labels)

# print 'accuracy using SVM one vs one: {}%'.format((100*correct)/len(prediction_labels))
# print("done in {}s".format(time.time() - t0))
# print ("making predictions")
    

In [8]:
# t0 = time.time()
# correct = 0
# incorrect = 0
# for i, image in enumerate(prediction_data):
#     pred = OneVsRestClassifier(clf.predict(image))
#     if pred == prediction_labels[i]:
#         correct += 1
#     else:
#         incorrect += 1
# print 'accuracy using SVM one vs rest: {}%'.format((100*correct)/(correct + incorrect))
# print("done in {}s".format(time.time() - t0))
    